In [1]:
# Libraries for interfacing with WFDB data
import wfdb
from wfdb import processing



#Classical libraries 
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import pandas as pd

from scipy.integrate import quad
from scipy.optimize import curve_fit
from numpy import trapz
from scipy.stats import norm



# importing detrending library
import obspy
from obspy.signal import detrend

In [5]:
paths = {'Fantasia':'physionet.org/files/fantasia/1.0.0/', 'MIT':'physionet.org/files/nsrdb/1.0.0/', 'Chfdb': 'physionet.org/files/chfdb/1.0.0/'}
print(paths['Fantasia'], type(paths['Fantasia']))

physionet.org/files/fantasia/1.0.0/ <class 'str'>


## This function its used to store the ECG data inside the original files into a Pandas Dataframe

In [ ]:
def import_ecg(path, files):
    
    '''
    Return ECG and RR_intervals dataframe.
    
    Inputs:
        path: path to the data's files
        files: files name without extentions
    
    outputs:
    return:
        ECG_sig: DataFrame with EGC signal for each file in each columns
        RR_intervals: DataFrame where in each columns are stored the interbeats intervals for each files
    '''


    # DataFrames to handle all signals (RAW ECG, RR intervals, Annotations, ...)
    ECG_sig=pd.DataFrame(columns=files)

    RR_intervals = pd.DataFrame(columns=files)

    # Read ECG data from files and store them in Pandas DataFrame
    # Save fields too, for further optional analysis
    signals_dfs=[]
    rr_dfs=[]

    if path == paths['Fantasia']: 
        for file in files:
                sig, fields =  wfdb.rdsamp(path + file, channels=[1]) # channel 1 maps to ECG data
                signals_dfs.append(pd.DataFrame({file: sig.T[0]}))

                xqrs = processing.XQRS(sig=-sig[:,0], fs=fields['fs'])
                xqrs.detect()
                rr=processing.calc_rr(xqrs.qrs_inds, fs=fields['fs'])
            
                rr_dfs.append(pd.DataFrame({file: rr}))

        ECG_sig=pd.concat(signals_dfs, ignore_index=True, axis=1)
        ECG_sig.columns=files

        RR_intervals=pd.concat(rr_dfs, ignore_index=True, axis=1)
        RR_intervals.columns=files
        
        return ECG_sig, RR_intervals
    
    else: 
         for file in files: 
              rr = processing.ann2rr(path+file, extension='ecg')
            rr_dfs.append(pd.DataFrame({file: rr}))
         
        RR_intervals=pd.concat(rr_dfs, ignore_index=True, axis=1)
        RR_intervals.columns=files
        return RR_intervals

    
    